###*Installing & Importing our dependicies*

In [ ]:
!git clone https://github.com/deepseek-ai/DeepSeek-Coder.git

In [ ]:
cd DeepSeek-Coder

In [ ]:
!pip install -r requirements.txt -q

In [ ]:
pip install -q python-docx fpdf

*Importing our depending used for our use case:*


*   Article Generation
*   Code Documentation and
*   Error Detection and Fixing of errors



In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
tokenizer = AutoTokenizer.from_pretrained("deepseek-ai/deepseek-coder-1.3b-instruct", trust_remote_code=True)
model = AutoModelForCausalLM. from_pretrained ("deepseek-ai/deepseek-coder-1.3b-instruct", trust_remote_code=True).cuda()


### *Building our article generation*

In [ ]:
!pip3 install azure-storage-blob azure-identity azure-cognitiveservices-vision-computervision azure-cognitiveservices-speech


In [ ]:
!wget http://archive.ubuntu.com/ubuntu/pool/main/o/openssl/libssl1.1_1.1.1f-1ubuntu2_amd64.deb
!sudo dpkg -i libssl1.1_1.1.1f-1ubuntu2_amd64.deb

In [ ]:
import random
import string
import requests
import uuid
import json
import azure.cognitiveservices.speech as speechsdk
import sys


def generate_article():
    """
    Generates articles with titles based on user input using a machine learning model.

    Returns:
    - str: Generated article based on user input.

    This function interacts with the user by accepting input through a chat-like interface ('User: ...').
    It utilizes a machine learning model (represented by 'model' and 'tokenizer') to generate articles
    based on the user's input. The 'max_length' parameter defines the maximum length of the generated article.
    """

    user_input = input('User: ')  # Prompt user for input
    language = input('What language do you want it in? ').lower()
    languages = []
    if language == 'english':
        languages = ['en']
    elif language == 'yoruba':
        languages.append('yo')
    elif language == 'igbo':
        languages.append('ig')
    elif language == 'hausa':
        languages.append('ha')
    else:
        languages.append('en')
    input_tokens = tokenizer(user_input, return_tensors='pt').to(model.device)

    # Generating article based on user input
    outputs = model.generate(
        **input_tokens,
        max_length=10678,
        temperature=1,
        top_p=0.7,
        num_beams=6,
        penalty_alpha=1,
        return_dict_in_generate=True,
        output_scores=True
    )

    # Decode the generated sequences to obtain the article
    generated_article = tokenizer.decode(outputs.sequences[0], skip_special_tokens=True)

    # Add your key and endpoint
    key = "2df6023bcb7445dd9ea3d38929bbb5e7"
    endpoint = "https://api.cognitive.microsofttranslator.com"
    location = "southafricanorth"

    path = '/translate'
    constructed_url = endpoint + path

    params = {
        'api-version': '3.0',
        'from': 'en',
        'to': languages
    }

    headers = {
        'Ocp-Apim-Subscription-Key': key,
        'Ocp-Apim-Subscription-Region': location,
        'Content-type': 'application/json',
        'X-ClientTraceId': str(uuid.uuid4())
    }

    body = [{
        'text': generated_article
    }]

    request = requests.post(constructed_url, params=params, headers=headers, json=body)
    response = request.json()

    return json.dumps(response[0]['translations'][0]['text'], sort_keys=True, ensure_ascii=False, indent=4, separators=(',', ': '))

def format_translated_article(translated_article):
    translated_article = translated_article.replace('\\n', '\n')
    paragraphs = translated_article.split('\n\n')

    formatted_article = ""
    for paragraph in paragraphs:
        if paragraph.startswith('**'):
            formatted_article += f"\n• {paragraph.replace('**', '')}\n"
        else:
            formatted_article += f"\n{paragraph}\n"

    return formatted_article

def speech_synthesis_to_mp3_file(formatted_article):
    """performs speech synthesis to a mp3 file"""
    # Creates an instance of a speech config with specified subscription key and service region.
    speech_config = speechsdk.SpeechConfig(subscription='23400053a3ea4672b9c1b0fb49ab7b62', region='southafricanorth')
    # Sets the synthesis output format.
    speech_config.set_speech_synthesis_output_format(speechsdk.SpeechSynthesisOutputFormat.Audio16Khz32KBitRateMonoMp3)
    # Creates a speech synthesizer using file as audio output.
    file_name = "outputaudio.mp3"
    file_config = speechsdk.audio.AudioOutputConfig(filename=file_name)
    speech_synthesizer = speechsdk.SpeechSynthesizer(speech_config=speech_config, audio_config=file_config)

    # Receives a text from console input and synthesizes it to mp3 file.
    while True:
        print("Enter some text that you want to synthesize, Ctrl-Z to exit")
        try:
            text = formatted_article
        except EOFError:
            break
        result = speech_synthesizer.speak_text_async(text).get()
        # Check result
        if result.reason == speechsdk.ResultReason.SynthesizingAudioCompleted:
            print("Speech synthesized for text [{}], and the audio was saved to [{}]".format(text, file_name))
        elif result.reason == speechsdk.ResultReason.Canceled:
            cancellation_details = result.cancellation_details
            print("Speech synthesis canceled: {}".format(cancellation_details.reason))
            if cancellation_details.reason == speechsdk.CancellationReason.Error:
                print("Error details: {}".format(cancellation_details.error_details))

def main():
    generated_article = generate_article()
    formatted_translated_article = format_translated_article(generated_article)
    speech_synthesis_to_mp3_file(formatted_translated_article)

    print("Formatted Translated Article:")
    print(formatted_translated_article)

if __name__ == "__main__":
    main()


### *Generating Documentation and Comments for Articles:*
*   This function aids in creating comprehensive documentation and comments for every line of code. Its purpose is to assist both current and new contributors to the codebase in comprehending legacy code. Moreover, it serves as a guide for identifying sections that might require updates or modifications.

In [ ]:
def generate_code_documentation(code, language, max_length):
    """
    Generates documentation and comments for the provided code snippet in the specified language.

    Args:
    - code (str): The code snippet to be documented.
    - language (str): The programming language used in the code snippet.
    - max_length (int): Maximum length of the generated documentation.

    Returns:
    - str: Generated documentation and comments for the code.

    This function utilizes a machine learning model -(Deepseek-coder) (represented by 'model' and 'tokenizer')
    to create comprehensive documentation and comments for the given code. It aims to clarify
    each line's purpose and functionality for future developers working on the codebase.
    Additionally, it takes into account the specified programming language for context.

    If necessary, the function could help align variable names with the function's goal,
    and it aims to improve the code's clarity and maintainability through better documentation.

    Example Usage:
    >>> generated_documentation = generate_code_documentation(code, 'Python', 500)
    """

    input_text = f"Please generate documentation and comments for the following {language.upper()} function/code. The goal is to clarify each line's purpose and functionality for future developers working on this codebase: {code}"
    inputs = tokenizer(input_text, return_tensors="pt").to(model.device)

    # Generating documentation and comments
    outputs = model.generate(**inputs,
                             max_length=10678,
                             temperature=0.8,
                             top_p=0.7,
                             num_beams=4,
                             penalty_alpha=1,
                             return_dict_in_generate=True,
                             output_scores=True)

    transition_scores = model.compute_transition_scores(
        outputs.sequences,
        outputs.scores,
        outputs.beam_indices,
        normalize_logits=False)

    # Decoding generated sequences to obtain documentation
    generated_documentation = tokenizer.decode(outputs.sequences[0], skip_special_tokens=True)

    return generated_documentation


In [ ]:
generate_code_documentation("""#include <iostream>

unsigned long long factorial(unsigned int n) {
    if (n == 0 || n == 1) {
        return 1;
    } else {
        return n * factorial(n - 1);
    }
}

int main() {
    unsigned int number = 5;
    unsigned long long result = factorial(number);
    std::cout << "Factorial of " << number << " is: " << result << std::endl;
    return 0;
}

""",'c++',800)

### *Identifying bugs and providing solutions for them:*
\
*   The primary objective of this function is to assist teams in enhancing their code quality by promptly detecting bugs and providing viable solutions to resolve them.








In [ ]:
def detect_and_fix_bugs(language, code_base):
    """
    Uses a machine learning model-(Deepseek-coder) to detect bugs and suggest fixes or improvements for the given code.

    Args:
    - language (str): The programming language used in the code.
    - code_base (str): The code snippet or codebase to be analyzed.

    Returns:
    - str: Generated suggestions for bug fixes or improvements.

    This function takes the provided code snippet in the specified programming language and
    utilizes a machine learning model (represented by 'model' and 'tokenizer') to identify
    any bugs or potential issues. It then suggests fixes or improvements to enhance the code's quality.

    Example Usage:
    >>> suggested_fixes = detect_and_fix_bugs('Python', 'def calculate_average(numbers): ...')
    """

    input_text = f'Please review the following {language} code and identify any bugs or potential issues. If you find any errors, please suggest a fix or improvements to the code: {code_base}'
    inputs = tokenizer(input_text, return_tensors="pt").to(model.device)

    # Generating suggestions for bug fixes or improvements
    outputs = model.generate(
        **inputs,
        max_length=3589,
        temperature=0.8,
        top_p=0.7,
        num_beams=4,
        penalty_alpha=1,
        return_dict_in_generate=True,
        output_scores=True
    )

    transition_scores = model.compute_transition_scores(
        outputs.sequences,
        outputs.scores,
        outputs.beam_indices,
        normalize_logits=False
    )

    # Decoding generated sequences to obtain suggestions
    generated_suggestions = tokenizer.decode(outputs.sequences[0], skip_special_tokens=True)

    return generated_suggestions


In [ ]:
detect_and_fix_bugs('python',"""def calculate_average(numbers):
    total = 0
    count = 0

    for num in numbers:
        total += num
        count += 1

    average = total / count  # Potential division by zero if 'count' is 0
    return average

# Test the function
values = [10, 20, 30, 40, 50]
result = calculate_average(values)
print("The average is:", result)
""")